### Neural Style Transfer on Face Images

By [Prithvi Raju](nprithviraj24@gmail.com) <br /> <br />

<break>
    
    
    
    
This Jupyter-notebook will discuss the methodology, which is not an original solution but encompasses my understanding of complications involved on Style transfer applied on faces, and way(s) to mitigate it.

##### NOTE: This notebook assumes that you are already familiar with Style Transfer.

### Example of Style Transfer on Face:
 <break>
 <break>
     
   Figure 1
![Style Transfer](style-transfer-on-face.png )

### Challenges in such type of Neural Style Transfer: 
    
    - Photo realism 
    - Semantic consistency.

The vanilla Style Transfer proposed by Gatys (2015) works perfectly on artistic image, but it fails to modify or synthesize realistic facial neural transfer as shown in the Figure 1. Style Trasnfer uses something called Gram matrix that is great for artistic purposes, but it suffers when we want to have some control over which textures are transferred to each part of the result. The reason behind this is it destroys the semantics of the style image, only preserving the basic texture components.  Human vision is very sensitive to facial irregularities and even small distortions can make a face look unrealistic.

Facial texture comprises skin texture details like wrinkles, pigmentation and pores, while facial structure consists of the meso-structures such as eyes, nose, mouth and face shape. Following example image shows the problems of style transfer applied on face, where the content image (Jim Carrey) and style image (Sméagol). The problem is even as the meso-structures from style image is contributing to the output image as well. In a way, this is not a good example of style transfer, where final output is not pleasing or adequate result.

![Meso-structure](meso-structure.png)

### Implementation details:  <br />

#### 1. Facial Semantic Regularization
 As discussed in the previous cell, the whole problem must be divided into two different solvable steps. One of them is to preserve the facial prior from the content image i.e. Facial Prior Regularization which ensures that there must be no changes to meso-structure of the face (content image) when it is trying to incorporate style from another image (style image). But this alone doesn't ensure that identity will be preserved, aditionally, during the training process we calculate Facial Semantic Structure Loss that tackles the challenge of preserving facial shape by minimizing a Facial Structure Loss which we define as an identity loss from a pre-trained face recognition network (VGG-Face) that implicitly preserves the facial structure. <br />


###### Facial Semantic Structure Loss

The idea is that we have to preserve the meso-structure of content image, and this is done using pretrained network VGG-Face. The mid level features (from VGG-Face) of an image captures the essence of only meso-structures of the given input face image, and then we minimize these representations with the one we want it to share the same representations. In a way we are enforcing the network to produce the same meso-structure as the content image, as VGG-Face's mid-level representations only captures the shapes and not the texture of the input face image.

#### 2. Enforcing style selection and localization.

 We have to nullify Gram Matrix as an option to synthesize style for output image because it gives global impact to the ouput image, instead we will have to look for other option such as CNN in MGAN that is ideal for preserving local textural structures. However, it also carries the semantic information from the style image, which violates the goal of preserving facial identity (discussed in previous point). For this, we augment the VGG-19 (used in MRF-CNN) framework with Facial semantic regularization.

#### Summarizing

In a nutshell, Facial identity is preserved using Facial Semantic Regularization, which regularizes the update of meso-structures using a facial prior and facial semantic structural loss. Texture loss regularizes the update of local textures from the style image. The output image is initialized with the content image and updated at each iteration by back-propagating the error gradients for the combined losses


#### Definitions inspired from Original Paper (FaceText)

###### Facial Prior Regularization

Inspired by the dropout regularization which randomly dropssome units and blocks the gradient during the optimization, we build a facial prior regularization that smoothly slows down the updating around the meso-structures. For generating the facial prior mask,we follow the prior work to generate 66 landmark points and draw contours for meso-structures. Then we build a landmark mask by applying a Gaussian blur to the facial contour and normalizethe output between 0 and 1, which provides a smooth transition between meso-structures and rest ofthe face. For implementation, we build a CNN layer that performs an identity mapping during theforward pass of the optimization, and scales the gradient with an element-wise product with the faceprior mask during back-propagation.
    


#### Architecture  (From FaceTex)

![FaceTex](FaceTex.png)

Please Note This methodology is understood from the original paper [FaceTex](https://arxiv.org/pdf/1706.04306.pdf)